In [1]:
# Install the dependencies if not installed
#!pip install --upgrade tensorflow-gpu=2.3.0

#!pip install nltk

#!pip install gensim

#!pip install spacy

#!pip install plotly

#!pip install pandas

#!pip install numpy

#!pip install matplotlib

#!pip install seaborn

#!pip install wordcloud

#!pip install sklearn

In [2]:
# Here we will import all required libraries

import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, TimeDistributed, LSTM, Embedding, RepeatVector
import tensorflow as tf

In [3]:
# Load the dataset
filename_en = './data/small_vocab_en.txt'
filename_fr = './data/small_vocab_fr.txt'
df_english = pd.read_csv(filename_en, sep='/t', names=['english'])
df_french = pd.read_csv(filename_fr, sep='/t', names=['french'])

/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
# Visualize english data
df_english

,english
0,"new jersey is sometimes quiet during autumn , ..."
1,the united states is usually chilly during jul...
2,"california is usually quiet during march , and..."
3,the united states is sometimes mild during jun...
4,"your least liked fruit is the grape , but my l..."
...,...
137855,"france is never busy during march , and it is ..."
137856,"india is sometimes beautiful during spring , a..."
137857,"india is never wet during summer , but it is s..."
137858,"france is never chilly during january , but it..."


In [5]:
# Visualize french data
df_french

,french
0,new jersey est parfois calme pendant l' automn...
1,les états-unis est généralement froid en juill...
2,"california est généralement calme en mars , et..."
3,"les états-unis est parfois légère en juin , et..."
4,"votre moins aimé fruit est le raisin , mais mo..."
...,...
137855,"la france est jamais occupée en mars , et il e..."
137856,"l' inde est parfois belle au printemps , et il..."
137857,"l' inde est jamais mouillé pendant l' été , ma..."
137858,"la france est jamais froid en janvier , mais i..."


In [6]:
# Check if there is any NULL data
print(df_english.isnull().sum())
print(df_french.isnull().sum())

english    0
dtype: int64
french    0
dtype: int64


In [7]:
# Concat both side by side for easy viewing
df = pd.concat([df_english, df_french], axis=1)
df

,english,french
0,"new jersey is sometimes quiet during autumn , ...",new jersey est parfois calme pendant l' automn...
1,the united states is usually chilly during jul...,les états-unis est généralement froid en juill...
2,"california is usually quiet during march , and...","california est généralement calme en mars , et..."
3,the united states is sometimes mild during jun...,"les états-unis est parfois légère en juin , et..."
4,"your least liked fruit is the grape , but my l...","votre moins aimé fruit est le raisin , mais mo..."
...,...,...
137855,"france is never busy during march , and it is ...","la france est jamais occupée en mars , et il e..."
137856,"india is sometimes beautiful during spring , a...","l' inde est parfois belle au printemps , et il..."
137857,"india is never wet during summer , but it is s...","l' inde est jamais mouillé pendant l' été , ma..."
137858,"france is never chilly during january , but it...","la france est jamais froid en janvier , mais i..."


In [8]:
def remove_punc(x):
    return re.sub('[!#?,.:";]', '', x)

In [9]:
df['english'] = df['english'].apply(remove_punc)
df['french'] = df['french'].apply(remove_punc)

In [10]:
# To get number of unique words

english_words = []
french_words = []

def get_unique(x, word_list):
    for word in x.split():
        if word not in word_list:
            word_list.append(word)
        
df['english'].apply(lambda x:get_unique(x,english_words))
df['french'].apply(lambda x:get_unique(x,french_words))

total_english_words = len(english_words)
total_french_words = len(french_words)

print("Number of unique english words ", total_english_words)
print("Number of unique english words ", total_french_words)

Number of unique english words  199
Number of unique english words  350


In [11]:
# Find max number of words in sentence so we can later do padding

maxlen_english = -1
for doc in df["english"]:
    #print(doc)
    tokens = nltk.word_tokenize(doc)
    if maxlen_english < len(tokens):
        maxlen_english = len(tokens)
        
print("Max number of words in any english sentence is ", maxlen_english)

maxlen_french = -1
for doc in df.french:
    tokens = nltk.word_tokenize(doc)
    if maxlen_french < len(tokens):
        maxlen_french = len(tokens)
        
print("Max number of words in any french sentence is ", maxlen_french)

Max number of words in any english sentence is  15
Max number of words in any french sentence is  23


In [12]:
type(df["english"])

pandas.core.series.Series

In [13]:
maxlen_english

15

In [14]:
# Create a tokenier to tokenize the words and create sequences of tokenized words
def tokenize_and_pad(x,maxlen):
    tokenizer = Tokenizer(char_level = False)
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(sequences, maxlen=maxlen, padding = 'post')
    return tokenizer, sequences, padded

In [15]:
x_tokenizer, x_sequences, x_padded = tokenize_and_pad(df.english, maxlen_english)
y_tokenizer, y_sequences, y_padded = tokenize_and_pad(df.french,  maxlen_french)

In [16]:
y_tokenizer

In [17]:
# Visualize example of a token
print("The tokenized version for english document\n", df.english[-1:].item(),"\n is : ", x_padded[-1:])

print("The tokenized version for french document\n", df.french[-1:].item(),"\n is : ", y_padded[-1:])

The tokenized version for english document
 the orange is her favorite fruit  but the banana is your favorite  
 is :  [[ 5 84  1 32 11 13  6  5 87  1 29 11  0  0  0]]
The tokenized version for french document
 l'orange est son fruit préféré  mais la banane est votre favori  
 is :  [[84  1 20 16 17  5  7 87  1 40 93  0  0  0  0  0  0  0  0  0  0  0  0]]


In [18]:
# Function to obtain text from padded
def pad_to_text(padded, tokenizer):
    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''
    temp = ' '.join([id_to_word[j] for j in padded])
    return temp

In [19]:
pad_to_text(y_padded[0], y_tokenizer)

"new jersey est parfois calme pendant l' automne et il est neigeux en avril         "

In [20]:
# Split data into train and test set
x_train, x_test, y_train, y_test = train_test_split(x_padded, y_padded, test_size=0.1)

In [21]:
# Total vocab size, since we added padding we add 1 to the total word count
english_vocab_size = total_english_words + 1
print("Complete English Vocab Size:", english_vocab_size)

french_vocab_size = total_french_words + 1
print("Complete French Vocab Size:", french_vocab_size)

Complete English Vocab Size: 200
Complete French Vocab Size: 351


In [24]:
#BUILDING THE MODEL
model = Sequential()

# Add an embedding layer
model.add(Embedding(english_vocab_size, 256, input_length = maxlen_english, mask_zero = True))

# Encoder
model.add(LSTM(256))

# decoder
# repeatvector repeats the input for the desired number of times to change 2D-array to 3D array
model.add(RepeatVector(maxlen_french))
model.add(LSTM(256, return_sequences = True))
model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 256)           51200     
                                                                 
 lstm (LSTM)                 (None, 256)               525312    
                                                                 
 repeat_vector (RepeatVector  (None, 23, 256)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 23, 256)           525312    
                                                                 
 time_distributed (TimeDistr  (None, 23, 351)          90207     
 ibuted)                                                         
                                                                 
Total params: 1,192,031
Trainable params: 1,192,031
Non-

In [25]:
# change the shape of target from 2D to 3D
y_train = np.expand_dims(y_train, axis = 2)
y_train.shape

(124074, 23, 1)

In [222]:
x_train_reshaped = np.expand_dims(x_train, axis = 2)

In [223]:
x_train_reshaped.shape

(124074, 15, 1)

In [ ]:
# Train the model
#with tf.device('/gpu:0'):
history = model.fit(x_train, y_train, batch_size=128, validation_split = 0.1, epochs=20)

Epoch 1/20
873/873 [==============================] - 380s 421ms/step - loss: 1.5381 - accuracy: 0.6278 - val_loss: 0.9653 - val_accuracy: 0.7163
Epoch 2/20
873/873 [==============================] - 388s 444ms/step - loss: 0.7353 - accuracy: 0.7815 - val_loss: 0.5638 - val_accuracy: 0.8324
Epoch 3/20
302/873 [=========>....................] - ETA: 4:27 - loss: 0.5224 - accuracy: 0.8437

In [26]:
model.save('./models/my_model.h5')

In [35]:
from tensorflow import keras
model = keras.models.load_model('./models/my_model.h5')

In [74]:
model

In [28]:
import matplotlib.pyplot as plt

In [31]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,26)
plt.plot(epochs, accuracy, 'g', label='accuracy')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [29]:
id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}

In [30]:
# function to make prediction
def prediction(x, x_tokenizer = x_tokenizer, y_tokenizer = y_tokenizer):
    predictions = model.predict(x)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    return ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])

In [31]:
# Let us take 5 examples
for i in range(5):
    print('Original English word - {}\n'.format(pad_to_text(x_test[i], x_tokenizer)))
    print('Original French word - {}\n'.format(pad_to_text(y_test[i], y_tokenizer)))
    print('Predicted French word - {}\n\n\n\n'.format(prediction(x_test[i:i+1])))

Original English word - they are going to india next may        

Original French word - ils vont en inde en mai prochain                

1/1 [==============================] - 2s 2s/step
Predicted French word - va va va va avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons




Original English word - paris is quiet during summer and it is cold in spring    

Original French word - paris est calme pendant l' été et il fait froid au printemps           

1/1 [==============================] - 0s 27ms/step
Predicted French word - grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand grand




Original English word - china is usually pleasant during january and it is usually mild in march  

Original French word - chine est généralement agréable en janvier et il est généralement doux en mars          

1/1 [==============================] - 0s 26ms/

In [35]:
# Save the weights separately
model.save_weights('Final_weights.h5')

#Save the model (including weights)
model.save('model.h5')

In [36]:
# To load to model we saved elsewhere
model1 = tf.keras.models.load_model('model.h5')

In [36]:
# To predict using the model we saved
# function to make prediction
def prediction(x, x_tokenizer = x_tokenizer, y_tokenizer = y_tokenizer):
    predictions = model.predict(x)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    return ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])

In [37]:
type(x_test[0:1])

numpy.ndarray

In [38]:
print('Predicted French word - {}\n\n\n\n'.format(prediction(x_test[0:1])))

1/1 [==============================] - 2s 2s/step
Predicted French word - va va va va avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons






## Prediction

In [39]:
sentence1 = ["the united states is sometimes quiet during autumn"]

In [40]:
def pad_to_text(padded, tokenizer):
    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''
    #temp = ' '.join([id_to_word[j] for j in padded])
    return id_to_word

In [41]:
def tokenize_and_pad1(x,maxlen=maxlen_english, df=df["english"]):
    tokenizer = Tokenizer(char_level = False)
    tokenizer.fit_on_texts(df)
    sequences = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(sequences, maxlen=maxlen, padding = 'post')
    return tokenizer, sequences, padded

In [42]:
tokenizer1, sequences1, padded1 = tokenize_and_pad1(sentence1)

In [43]:
print(tokenizer1)
print(sequences1)
print(padded1)

[[5, 20, 21, 1, 8, 67, 4, 39]]
[[ 5 20 21  1  8 67  4 39  0  0  0  0  0  0  0]]


In [44]:
def pad_to_text(padded, tokenizer):
    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''
    #temp = ' '.join([id_to_word[j] for j in padded])
    return id_to_word

In [45]:
sentence = ["the united states is sometimes quiet during autumn"]

In [46]:
print('Predicted French word - {}\n\n\n\n'.format(prediction(padded1)))

1/1 [==============================] - 0s 29ms/step
Predicted French word - calme calme elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle elle






In [47]:
sent2 = ["new jersey is sometimes quiet during autumn  and it is snowy in april"]

In [48]:
tokenizer2, sequences2, padded2 = tokenize_and_pad1(sent2)

In [49]:
print('Predicted French word - {}\n\n\n\n'.format(prediction(padded2)))

1/1 [==============================] - 0s 30ms/step
Predicted French word - aux ses ses ses tour tour tour tour tour tour tour tour tour tour tour tour tour tour tour tour tour tour tour






In [53]:
def preprocessing_prediction(x,maxlen=maxlen_english, df=df["english"]):
    tokenizer = Tokenizer(char_level = False)
    tokenizer.fit_on_texts(df)
    sequences = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(sequences, maxlen=maxlen, padding = 'post')
    predictions = model.predict(padded)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    pred= ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])
    return pred

In [54]:
sent1 = ["india is sometimes busy during august but it is wonderful in fall "]

In [55]:
preprocessing_prediction(sent1,maxlen=maxlen_english, df=df["english"])

1/1 [==============================] - 0s 26ms/step


'aller aller aller aller aller aller aller était était était était était était était était était était était était était était était était'

### seq2seq

In [67]:
from keras.layers import Input
from keras.layers import Embedding

In [72]:
def seq2seq_model_builder(HIDDEN_DIM=300):
    
    encoder_inputs = Input(shape=(maxlen_english, ), dtype='int32',)
    encoder_embedding = Embedding(encoder_inputs, output_dim= 20)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
    decoder_inputs = Input(shape=(maxlen_english, ), dtype='int32',)
    decoder_embedding = embed_layer(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    
    return model

In [73]:
model = seq2seq_model_builder(HIDDEN_DIM=300)
model.summary()

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

### RNN

input_NLP: 
- padded 
- tokenizer
- sequences

In [129]:
!pip install helper 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 15.3 MB/s eta 0:00:00m eta 0:00:01
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19166 sha256=197e621cc0a6f82b11dfdefc0c32295b2b5f278656d4731145ede8d500a78806
  Stored in directory: /home/wiem/snap/jupyter/6/.cache/pip/wheels/41/b4/cf/a065acf9ad15025a24e687f5cfddecc4f22f28e0844156ce69
Successfully built helper


In [ ]:
tokenizer1, sequences1, padded1 = tokenize_and_pad1(sentence1)

In [144]:
padded1.shape

(1, 15)

In [157]:
from keras.layers import TimeDistributed, Dropout
from keras.optimizers import Adam

In [165]:
x_padded.shape

(137860, 15)

In [193]:
french_vocab_size

351

In [186]:
learning_rate = 0.005

    
# TODO: Build the layers
model = Sequential()
model.add(GRU(256,return_sequences=True, input_shape=x_train.shape))
model.add(TimeDistributed(Dense(1024, activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

In [187]:
# Compile model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])

In [188]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_16 (GRU)                (None, 124074, 256)       209664    
                                                                 
 time_distributed_20 (TimeDi  (None, 124074, 1024)     263168    
 stributed)                                                      
                                                                 
 dropout_9 (Dropout)         (None, 124074, 1024)      0         
                                                                 
 time_distributed_21 (TimeDi  (None, 124074, 351)      359775    
 stributed)                                                      
                                                                 
Total params: 832,607
Trainable params: 832,607
Non-trainable params: 0
_________________________________________________________________


In [189]:
x_train.shape

(124074, 15)

In [202]:
x_train.shape

(124074, 15)

In [204]:
y_train.shape

(124074, 23, 1)

In [194]:
history = model.fit(x_train, y_train, batch_size=128, validation_split = 0.1, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/input_spec.p

In [168]:
learning_rate = 0.003

# Build the layers    
model = Sequential()
# Embedding
model.add(Embedding(english_vocab_size, 128, input_length=maxlen_english, input_shape=x_padded.shape))
# Encoder
model.add(Bidirectional(GRU(128)))
model.add(RepeatVector(output_sequence_length))
# Decoder
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(TimeDistributed(Dense(512, activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])

ValueError: Input 0 of layer "bidirectional_3" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 137860, 15, 128)

# Simple RNN

In [220]:
x_train.shape

(124074, 15)

In [221]:
y_train.shape

(124074, 23, 1)

In [215]:
from tensorflow.keras import layers

In [226]:
model_RNN = keras.Sequential()
model_RNN.add(Embedding(english_vocab_size, 256, input_length = maxlen_english, mask_zero = True))
model_RNN.add(layers.SimpleRNN(128))
model_RNN.add(layers.Dense(10))
#model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
model_RNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_RNN.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 15, 256)           51200     
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 128)               49280     
                                                                 
 dense_31 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [227]:
history = model_RNN.fit(x_train_reshaped, y_train, batch_size=128, validation_split = 0.1, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/wiem/snap/jupyter/common/lib/python3.7/s